<mark>***Import the Files***<?mark>

In [3]:
import pandas as pd

# Load the datasets
W_df = pd.read_csv("C:\\Users\\PC\\Desktop\\Delhi _weather_data.csv")
H_df = pd.read_csv("C:\\Users\\PC\\Desktop\\Holiday_calendar.csv")
T_df = pd.read_csv("C:\\Users\\PC\\Desktop\\Dataset_Uber_Traffic.csv")


<mark>***Basic Cleaning Before Merging***</mark>

***Convert Date Columns to Standard Format***

In [6]:
# Convert weather dataset datetime column
W_df['datetime'] = pd.to_datetime(W_df['datetime'], format='%d-%m-%Y')

# Convert holiday dataset date column
H_df['Date'] = pd.to_datetime(H_df['Date'], format='%d-%m-%Y')

# Convert traffic dataset DateTime column
T_df['DateTime'] = pd.to_datetime(T_df['DateTime'], format='%d/%m/%y %H:%M')

# Extract only the date from traffic and weather datasets for merging
T_df['Date'] = T_df['DateTime'].dt.date
W_df['Date'] = W_df['datetime'].dt.date


***Handle Missing Values (Before Merging)***

In [9]:
# Fill missing values safely
W_df.loc[:, 'preciptype'] = W_df['preciptype'].fillna("None")

# Drop 'severerisk' column safely
if 'severerisk' in W_df.columns:
    W_df = W_df.drop(columns=['severerisk'])

# Remove duplicates safely
W_df = W_df.drop_duplicates()
H_df = H_df.drop_duplicates()
T_df = T_df.drop_duplicates()


<mark>***Merge the Datasets***</mark>

***Merge Traffic & Weather Data***

In [14]:
merged_df = pd.merge(T_df, W_df, on='Date', how='left')


***Merge Holiday Data***

In [17]:
# Convert 'Date' column in merged dataset to datetime before merging
merged_df['Date'] = pd.to_datetime(merged_df['Date'])

# Merge with holiday dataset
merged_df = pd.merge(merged_df, H_df, on='Date', how='left')


<mark>***Final Cleaning After Merging***</mark>

***Check for Missing Values***

In [21]:
print(merged_df.isnull().sum())  # Check missing values


DateTime                0
Junction                0
Vehicles                0
ID                      0
Date                    0
name                    0
datetime                0
tempmax                 0
tempmin                 0
temp                    0
feelslikemax            0
feelslikemin            0
feelslike               0
dew                     0
humidity                0
precip                  0
precipprob              0
precipcover             0
preciptype              0
snow                    0
snowdepth               0
windgust                0
windspeed               0
winddir                 0
sealevelpressure        0
cloudcover              0
visibility              0
solarradiation          0
solarenergy             0
uvindex                 0
sunrise                 0
sunset                  0
moonphase               0
conditions              0
description             0
icon                    0
stations                0
Holiday             45360
dtype: int64

***Fill or Drop Missing Values***

In [24]:
# Fill missing values safely
merged_df = merged_df.assign(Holiday=merged_df['Holiday'].fillna("No Holiday"))

# Use ffill() correctly
merged_df = merged_df.ffill()  # Forward fill missing values


<mark>***Final Data Verification***</mark>

In [27]:
print(merged_df.head())  # Check the first few rows
print(merged_df.info())  # Check data types and missing values


             DateTime  Junction  Vehicles           ID       Date   name  \
0 2015-11-01 00:00:00         1        15  20151101001 2015-11-01  Delhi   
1 2015-11-01 01:00:00         1        13  20151101011 2015-11-01  Delhi   
2 2015-11-01 02:00:00         1        10  20151101021 2015-11-01  Delhi   
3 2015-11-01 03:00:00         1         7  20151101031 2015-11-01  Delhi   
4 2015-11-01 04:00:00         1         9  20151101041 2015-11-01  Delhi   

    datetime  tempmax  tempmin  temp  ...  solarenergy  uvindex  \
0 2015-11-01     30.2     17.7  23.5  ...         17.2        7   
1 2015-11-01     30.2     17.7  23.5  ...         17.2        7   
2 2015-11-01     30.2     17.7  23.5  ...         17.2        7   
3 2015-11-01     30.2     17.7  23.5  ...         17.2        7   
4 2015-11-01     30.2     17.7  23.5  ...         17.2        7   

               sunrise               sunset  moonphase  conditions  \
0  2015-11-01T06:32:39  2015-11-01T17:36:22       0.66       Clear   


<mark>***Handle data quality issues***</mark>

***Remove Duplicates***

In [31]:
# Remove duplicates from the merged dataset
merged_df = merged_df.drop_duplicates()


***Handle Missing Values (Imputation or Removal)***

In [34]:
print(merged_df.isnull().sum())  # Display missing values


DateTime            0
Junction            0
Vehicles            0
ID                  0
Date                0
name                0
datetime            0
tempmax             0
tempmin             0
temp                0
feelslikemax        0
feelslikemin        0
feelslike           0
dew                 0
humidity            0
precip              0
precipprob          0
precipcover         0
preciptype          0
snow                0
snowdepth           0
windgust            0
windspeed           0
winddir             0
sealevelpressure    0
cloudcover          0
visibility          0
solarradiation      0
solarenergy         0
uvindex             0
sunrise             0
sunset              0
moonphase           0
conditions          0
description         0
icon                0
stations            0
Holiday             0
dtype: int64


***Correct Data Inconsistencies***

In [37]:
#Convert Date Columns to Consistent Format
merged_df['Date'] = pd.to_datetime(merged_df['Date'])


In [39]:
#Fix Temperature Anomalies
merged_df = merged_df[(merged_df['temp'] >= -10) & (merged_df['temp'] <= 50)]


<mark>***Normalize or Standardize Data***</mark>

***Normalize Data (Rescale to 0-1)***

In [43]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
merged_df[['temp', 'humidity', 'windspeed']] = scaler.fit_transform(merged_df[['temp', 'humidity', 'windspeed']])


***Standardize Data (Mean = 0, Std = 1)***

In [46]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
merged_df[['temp', 'humidity', 'windspeed']] = scaler.fit_transform(merged_df[['temp', 'humidity', 'windspeed']])


<mark>***Final Data Check***</mark>

In [82]:
print(merged_df.info())  # Check data types
print(merged_df.describe())  # Summary statistics


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48120 entries, 0 to 48119
Data columns (total 38 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   DateTime          48120 non-null  datetime64[ns]
 1   Junction          48120 non-null  int64         
 2   Vehicles          48120 non-null  int64         
 3   ID                48120 non-null  int64         
 4   Date              48120 non-null  datetime64[ns]
 5   name              48120 non-null  object        
 6   datetime          48120 non-null  datetime64[ns]
 7   tempmax           48120 non-null  float64       
 8   tempmin           48120 non-null  float64       
 9   temp              48120 non-null  float64       
 10  feelslikemax      48120 non-null  float64       
 11  feelslikemin      48120 non-null  float64       
 12  feelslike         48120 non-null  float64       
 13  dew               48120 non-null  float64       
 14  humidity          4812

In [ ]:
merged_df['FinalDateTime'] = merged_df['DateTime'].combine_first(merged_df['Date']).combine_first(merged_df['datetime'])


In [ ]:
merged_df.drop(columns=['DateTime', 'Date', 'datetime'], inplace=True)


In [82]:
#rearrenge the columns
if 'FinalDateTime' in merged_df.columns:
    # Reorder columns
    cols = ['FinalDateTime'] + [col for col in merged_df.columns if col != 'FinalDateTime']
    merged_df = merged_df[cols]


In [86]:
merged_df['preciptype'] = merged_df['preciptype'].fillna("No Precipitation")


In [90]:
merged_df.to_excel(r"C:\\Users\\PC\\Desktop\\merged_data.xlsx", index=False)
